In [ ]:
from selenium import webdriver
driverPath = 'edgedriver_win64\msedgedriver.exe'
import time
wd = webdriver.Edge(driverPath)

wd.get('https://yb.dhu.edu.cn/sccloudV2/NewStudent1/User/HandBook/handBookTestStart?customer_id=5&module_tag=HANDBOOK')

wd.find_element_by_xpath('/html/body/main/section[1]/div/div[1]/input').send_keys('输入用户名') #输入用户名
wd.find_element_by_xpath('/html/body/main/section[1]/div/div[2]/input').send_keys('输入密码') #输入密码
wd.find_element_by_xpath('/html/body/main/section[1]/div/div[4]/button[1]').click() #点击登陆


In [8]:
# wd.get('https://yb.dhu.edu.cn/sccloudV2/NewStudent1/User/HandBook/handBookTestDetail?customer_id=5&module_tag=HANDBOOK') #学生手册
# wd.get('https://yb.dhu.edu.cn/sccloudV2/NewStudent1/User/HandBook/handBookTestDetail?customer_id=5&module_tag=NETWORK_SECURITY') #网络安全
wd.get('https://yb.dhu.edu.cn/sccloudV2/NewStudent1/User/HandBook/handBookTestDetail?customer_id=5&module_tag=GARBAGE_CG') #垃圾分类

In [9]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

for count in range(0,75):
    soup = bs(wd.page_source) 
    test_content = soup.find('div',class_='test_content')
    form_data = {
        "quiz_question_id":test_content.get('data-id'),
        "question_type_fixed" : test_content.get('data-question-type-fixed'),
        "user_quiz_answer_id": '237042', #随便填
        "quiz_id" : '490',#489学生手册 #491网络安全 #490垃圾分类
        "answer" : 'A',
    }
    try:
        dataStr = "user_quiz_answer_id=" + form_data ['user_quiz_answer_id']+"&quiz_id="+form_data['quiz_id'] + "&question_type_fixed=" +form_data['question_type_fixed'] + "&answer=" + form_data['answer'] + "&quiz_question_id=" + form_data['quiz_question_id']
    except TypeError:
        break
    # print(dataStr)
    id_list = soup.findAll('li',class_='answer-click') 
    id_dic = {}
    num = 1
    for item in id_list:
        id_dic[item.get('data-id')] = str(num)
        num+=1
    # print(id_dic) 
    jq = ''
    with open('jquery.min.js') as f:
        jq = f.read()
    wd.execute_script(jq)
    ajax_query = '''
    $.ajax({type:"POST",url:"/sccloudV2/index.php/NewStudent1/User/HandBookAjax/addStudentAnswer",
    data: "%s",
    async: false,
    dataType:"json",
    success: function () {}
    })
            '''%(dataStr)
    ajax_query = ajax_query.replace(" ", "").replace("\n", "")
    # print(ajax_query)
    resp = wd.execute_script("return " + ajax_query)
    answer = []
    for asw in resp['data']['right_answer_list']:
        answer.append(asw['mw_quiz_question_option_id'])
    for a in answer:
        n = '/html/body/div[1]/div[3]/div[1]/ul/li[%s]'%id_dic[a]
        wd.find_element(By.XPATH,n).click()
    wd.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[2]').click()
    time.sleep(1.5)

